<a href="https://colab.research.google.com/github/upamasharmin/Llama-Series/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"upamasharmin","key":"32bedd56494a9fb7e105b5d71c415a82"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
#API to fetch the dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 93% 75.0M/80.9M [00:00<00:00, 155MB/s]
100% 80.9M/80.9M [00:00<00:00, 149MB/s]


In [5]:
#dataset extraction
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Datset extracted')

Datset extracted


In [6]:
#importing libraries
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding= 'ISO 8859-1')

In [9]:
twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#creating column names
column_names = ['target','id','date','flag','user','text']
twitter_data.columns = column_names

In [12]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [13]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [14]:
twitter_data['target'].value_counts()

,count
target,
4,800000
0,799999


In [15]:
#converting target '4' to '1'
twitter_data.replace({'target':{4:1}},inplace =True)
twitter_data['target'].value_counts()

,count
target,
1,800000
0,799999


In [16]:
#stemming
port_stem = PorterStemmer()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content =[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [17]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [18]:
print(twitter_data['stemmed_content'])

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1599999, dtype: object


In [19]:
#separating data and labels
X= twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [20]:
print(X)
print(Y)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


In [22]:
#splitting to train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state=2)

In [23]:
print(X_train.shape, Y_train.shape)

(1279999,) (1279999,)


In [24]:
#converting from text to numeric data
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X_train =vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [25]:
print(X_train)

  (0, 109298)	0.37470892870977707
  (0, 185125)	0.5296934860731303
  (0, 234920)	0.4196620311217523
  (0, 354566)	0.358082129508793
  (0, 436655)	0.27247657460069113
  (0, 443035)	0.44770025018080273
  (1, 19762)	0.25286013616648123
  (1, 86730)	0.33015139495167095
  (1, 119689)	0.4309123985667578
  (1, 152015)	0.49196931362728563
  (1, 166826)	0.3977368489124923
  (1, 172343)	0.3785321144028949
  (1, 319721)	0.3128279791558143
  (2, 93826)	0.2233049434503089
  (2, 117914)	0.3986115507964782
  (2, 124728)	0.33175938915396974
  (2, 146003)	0.2213048673720909
  (2, 220710)	0.2626149875630723
  (2, 279017)	0.2849974993622606
  (2, 326288)	0.5853283798404599
  (2, 412554)	0.28961919890983556
  (2, 445635)	0.23563700521896036
  (3, 46285)	0.6887696820070169
  (3, 93826)	0.46202690964703447
  (3, 331805)	0.4656145237310222
  :	:
  (1279996, 85239)	0.34250398106721314
  (1279996, 295620)	0.47908235635831214
  (1279996, 299643)	0.5685438365651838
  (1279997, 283376)	0.3478335165480973
  (12799

In [26]:
#training model
model = LogisticRegression(max_iter =1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [27]:
#accuracy score of the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [31]:
print('Accuracy data :',training_data_accuracy)

Accuracy data : 0.801063125830567


In [32]:
X_test_prediction = model.predict(X_test)
training_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [33]:
print('Accuracy data :',training_data_accuracy)

Accuracy data : 0.7771625


In [35]:
#saving the model
import pickle

In [36]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))


In [37]:
#loading the saved model
loaded_model =pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new = X_test[200]

prediction= model.predict(X_new)